# 理解 Embedding层 和 Liear层 的不同

- 在Pytorch中Embedding层和linear层一样实序列矩阵乘法操作；我们使用embedding层的原因是其计算效率更高
- 我们使用使用Pytorch代码示例一步一步观察其中的关系和细节

In [1]:
import torch

print("PyTorch version:", torch.__version__)

PyTorch version: 2.2.2+cu121


## 使用 nn.Embedding

In [2]:
# 假设我们有如下3个训练样例
# 可能代表了在一个LLM场景中的 token IDs
idx = torch.tensor([2, 3, 1])

# embedding矩阵的行数可以由最大 token ID + 1 获取
# 如果最大token ID为3，就需要4行，对应的可能的IDs为0，1，2，3
num_idx = max(idx) + 1

# 所期望的embedding维度是一个超参数
out_dim = 5

- 让我们实现一个简单的embedding层

In [3]:
# 因为embedding层的权重由一些较小的随机值初始化，我们使用随机化种子用于复现
torch.manual_seed(123)

embedding = torch.nn.Embedding(num_idx, out_dim)

In [4]:
embedding.weight

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  1.5810],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015],
        [ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953]], requires_grad=True)

- 我们可以使用embedding层获取ID为1的训练样本的向量表示

In [5]:
embedding(torch.tensor([1]))

tensor([[ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 如下是刚才发生的可视化：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/1.png" width="400px">

- 相似地，我们可以获取ID为2的向量表示：

In [6]:
embedding(torch.tensor([2]))

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315]],
       grad_fn=<EmbeddingBackward0>)

- 现在，让我们将所有的训练样本进行转换：

In [7]:
idx = torch.tensor([2, 3, 1])
embedding(idx)

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/3.png" width="450px">

## 使用 nn.Linear

- 现在，我们将证明上述嵌入层在 PyTorch 的独热编码表示上与 `nn.Linear` 层的效果完全相同
- 首先，让我们将token IDs转化为一个独热表示

In [8]:
onehot = torch.nn.functional.one_hot(idx)
onehot

tensor([[0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0]])

- 接下来，我们初始化一个Linear层，用于执行 $X W^\top$ 的矩阵乘法

In [9]:
torch.manual_seed(123)
linear = torch.nn.Linear(num_idx, out_dim, bias=False)
linear.weight

Parameter containing:
tensor([[-0.2039,  0.0166, -0.2483,  0.1886],
        [-0.4260,  0.3665, -0.3634, -0.3975],
        [-0.3159,  0.2264, -0.1847,  0.1871],
        [-0.4244, -0.3034, -0.1836, -0.0983],
        [-0.3814,  0.3274, -0.1179,  0.1605]], requires_grad=True)

- 请注意，PyTorch 中的线性层也是用小随机权重初始化的；要将它与上面的 "嵌入 "层直接比较，我们必须使用相同的小随机权重，这就是我们在这里重新分配权重的原因：

In [10]:
linear.weight = torch.nn.Parameter(embedding.weight.T.detach())

- 现在我们可以将此linear层作用于独热编码表示的输入上：

In [11]:
linear(onehot.float())

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]], grad_fn=<MmBackward0>)

- 我们可以看到，这与我们使用embedding层得到的一模一样：

In [12]:
embedding(idx)

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 对于第一个训练样例的token ID，会进行如下计算：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/4.png" width="450px">

- 对第二个token ID：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/5.png" width="450px">

- 由于每个独热编码行中除了一个索引以外的所有索引都是0(根据设计)，因此这个矩阵乘法本质上与查找单热元素相同
- 在独热编码上使用矩阵乘法相当于嵌入层查找，但如果我们使用大型嵌入矩阵，效率可能会很低，因为有很多浪费的零乘法